Full code to make all waypoints from road sections 

In [1]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
import glob
import os
try:
    sys.path.append(glob.glob('./carla/dist/carla-0.9.14-py3.7-win-amd64.egg')[0])
except IndexError:
    print('Couldn\'t import Carla egg properly')
import carla

try:
    sys.path.insert(0,r'C:\Users\ashokkumar\source\repos\AD\carla')
except IndexError:
    pass

from carla import ColorConverter as cc
from agents.navigation.behavior_agent import BehaviorAgent  
from agents.navigation.basic_agent import BasicAgent  

# connect to the sim 
client = carla.Client('localhost', 2000)

world = client.get_world()
world = client.load_world('Town02')

spectator = world.get_spectator()

# get map look at the map
town_map = world.get_map()
roads = town_map.get_topology()

# set up route generations

In [2]:
'''
this is a better option:
1. it allows to set a distance between waypoints
2. it gets them all without gaps
3. Do not need ti use route planner

#this takes about 4 minutes - in RL context, this only needs to run at the start or when you change a towns
'''
all_waypoints = town_map.generate_waypoints(1) #note distance between waypoints in meters

# make unique
unique_waypoints = []
for wp in all_waypoints:
    if len(unique_waypoints)==0:
        unique_waypoints.append(wp) #first waypoint is added regardless to start the list
    else:
        found = False
        for uwp in unique_waypoints: #check for same located waypoints and ignore if found
            if abs(uwp.transform.location.x - wp.transform.location.x) < 0.1 \
                            and abs(uwp.transform.location.y - wp.transform.location.y)<0.1 \
                            and abs(uwp.transform.rotation.yaw - wp.transform.rotation.yaw)<20:  #this checks same direction
                found = True
                break
        if not found:
            unique_waypoints.append(wp)

# draw all point in the sim for 60 seconds
for wp in unique_waypoints:
    world.debug.draw_string(wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)

#move spectator for top down view to see all points 
spectator_pos = carla.Transform(carla.Location(x=0,y=30,z=200), carla.Rotation(pitch=-90, yaw=0, roll=0))
spectator.set_transform(spectator_pos)

In [3]:
'''
now we will see how this will be used in real examples, like during RL training
e.g. see how long it takes to find closest point to the car

'''
#remove any cars/clean up the sim
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
blueprint_library = world.get_blueprint_library()

# Now let's filter all the blueprints of type 'vehicle' and choose one
bp = blueprint_library.find('vehicle.tesla.model3')
transform = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(bp, transform)  
spectator = world.get_spectator()
spectator_transform =  vehicle.get_transform()
spectator_transform.location += carla.Location(z = 10.0)
world.get_spectator().set_transform(spectator_transform)


agent = BasicAgent(vehicle)
waypoint_counter = 0


In [4]:
my_waypoint = vehicle.get_transform().location
curr_distance = 1000
for wp in unique_waypoints:
    dist = my_waypoint.distance(wp.transform.location)
    if dist < curr_distance:
        curr_distance =  dist
        selected_wp = wp

next_wp = selected_wp.next(5.0)[0]
destination = next_wp.transform.location
agent.set_destination(destination)

In [5]:
while True:
    if agent.done():
        next_wp = selected_wp.next(5.0)[0]
        destination = next_wp.transform.location
        agent.set_destination(destination)
        world.debug.draw_string(next_wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=255, b=0), life_time=2.0,
        persistent_lines=True)
        print("The target has been reached, setting next waypoint")
        selected_wp = next_wp
        waypoint_counter += 1

    if waypoint_counter > 100:
        print("We have reached the end of the waypoints")
        break
    vehicle.apply_control(agent.run_step())

The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, setting next waypoint
The target has been reached, se

In [7]:
vehicle_pos = vehicle.get_transform().location
print("The vehicle pos: ", vehicle_pos)
next_wp = selected_wp.next(5.0)[0]
print("The next waypoint: ", next_wp.transform.location)


The vehicle pos:  Location(x=123.497292, y=306.554932, z=0.221790)
The next waypoint:  Location(x=136.511978, y=306.559784, z=0.000000)


In [11]:
def get_lateral_distance(vehicle, waypoint, max_distance=8.0):
    # Get vehicle and waypoint positions
    vehicle_location = vehicle.get_transform().location
    waypoint_location = waypoint.transform.location

    # Get the forward vector of the waypoint (direction the road is facing)
    waypoint_forward_vector = waypoint.transform.get_forward_vector()

    # Calculate the vector from the waypoint to the vehicle
    vehicle_vector = vehicle_location - waypoint_location

    # Convert the CARLA location objects to NumPy arrays for easier vector math
    waypoint_forward_np = np.array([waypoint_forward_vector.x, waypoint_forward_vector.y])
    vehicle_vector_np = np.array([vehicle_vector.x, vehicle_vector.y])

    # Normalize the waypoint's forward vector to get the direction
    waypoint_forward_np = waypoint_forward_np / np.linalg.norm(waypoint_forward_np)

    # Project the vehicle vector onto the waypoint's forward vector
    projection_length = np.dot(vehicle_vector_np, waypoint_forward_np)

    # Get the projection point (the closest point on the waypoint's direction line)
    projection_point = projection_length * waypoint_forward_np

    # Calculate the lateral distance as the distance between the vehicle's position and the projection point
    lateral_vector = vehicle_vector_np - projection_point
    lateral_distance = np.linalg.norm(lateral_vector)

    # Normalize the lateral distance to the range [-1, 1]
    normalized_lateral_distance = np.clip(lateral_distance / max_distance, -1, 1)

    return normalized_lateral_distance, lateral_distance

norm_lateral_distance, lateral_distance = get_lateral_distance(vehicle, next_wp)
print("The lateral distance is: ", lateral_distance)
print("The normalized lateral distance is: ", norm_lateral_distance)

The lateral distance is:  0.0004417125493269432
The normalized lateral distance is:  5.52140686658679e-05


In [19]:
def get_relative_heading(vehicle_transform, waypoint_transform):
    # Extract vehicle and waypoint yaw in degrees
    vehicle_yaw = vehicle_transform.rotation.yaw
    waypoint_yaw = waypoint_transform.rotation.yaw
    
    # Compute relative heading (difference between vehicle and waypoint yaw)
    relative_heading = (vehicle_yaw - waypoint_yaw) % 360
    
    # Normalize to [-180, 180] range
    if relative_heading > 180:
        relative_heading -= 360
    normalized_heading = relative_heading / 180.0
    return np.clip(normalized_heading, -1, 1)

print('the relateve heading is:', get_relative_heading(vehicle.get_transform(), next_wp.transform))


the relateve heading is: -3.400500863790512e-05


In [18]:

def get_future_headings(vehicle_transform, waypoints):
    headings = []
    for waypoint in waypoints[:3]:  # Get the heading to the next 3 waypoints
        heading = get_relative_heading(vehicle_transform, waypoint.transform)
        headings.append(heading/180)
    return headings  # Return a list of normalized headings
print('the future headings are:', get_future_headings(vehicle.get_transform(), unique_waypoints))


the future headings are: [0.0027809961959374724, -0.002774559359618083, 0.0027809961959374724]


In [20]:
for actor in world.get_actors().filter('*vehicle*'):
    print(actor)

Actor(id=135, type=vehicle.tesla.model3)


: 